In [34]:
import tensorflow as tf
from tensorflow import data
import shutil
import math
from datetime import datetime
from tensorflow.python.feature_column import feature_column

from tensorflow.contrib.learn import learn_runner
from tensorflow.contrib.learn import make_export_strategy

print(tf.__version__)

1.4.0


## Steps to use the TF Estimator APIs
1. Define dataset **metadata**
2. Define **data input function** to read the data from the source (.csv) + **apply pre-processing**
3. Create TF **feature columns** based on metadata + **extended feature columns**
4. Instantiate an **estimator** with the required **feature columns & parameters**
5. Define a **serving function**
6. Run **Experiment** by supplying training and validation data, as well as required parameters
7. **Evaluate** the model using test data
8. Perform **predictions**

In [2]:
train_data_files = ['data/train-data.csv']
valid_data_files = ['data/valid-data.csv']
test_data_files = ['data/test-data.csv']

model_name = 'reg-model-03'

resume = False
train = True
preprocess = True

## 1. Define Dataset Metadata

In [3]:
HEADER = ['key','x','y','alpha','beta','target']
HEADER_DEFAULTS = [[0], [0.0], [0.0], ['NA'], ['NA'], [0.0]]

NUMERIC_FEATURE_NAMES = ['x', 'y']  

CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY = {'alpha':['ax01', 'ax02'], 'beta':['bx01', 'bx02']}
CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.keys())

FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES

TARGET_NAME = 'target'

UNUSED_FEATURE_NAMES = list(set(HEADER) - set(FEATURE_NAMES) - {TARGET_NAME})

print("Header: {}".format(HEADER))
print("Numeric Features: {}".format(NUMERIC_FEATURE_NAMES))
print("Categorical Features: {}".format(CATEGORICAL_FEATURE_NAMES))
print("Target: {}".format(TARGET_NAME))
print("Unused Features: {}".format(UNUSED_FEATURE_NAMES))

Header: ['key', 'x', 'y', 'alpha', 'beta', 'target']
Numeric Features: ['x', 'y']
Categorical Features: ['alpha', 'beta']
Target: target
Unused Features: ['key']


## 2. Define Data Input Function

### a. parsing and preprocessing logic

In [4]:
def parse_csv_row(csv_row):
    
    columns = tf.decode_csv(csv_row, record_defaults=HEADER_DEFAULTS)
    features = dict(zip(HEADER, columns))
    
    for column in UNUSED_FEATURE_NAMES:
        features.pop(column)
    
    target = features.pop(TARGET_NAME)

    return features, target

def process_features(features):
    
    if preprocess:
        features["x_2"] = tf.square(features['x'])
        features["y_2"] = tf.square(features['y'])
        features["xy"] = tf.multiply(features['x'], features['y']) # features['x'] * features['y']
        features['dist_xy'] =  tf.sqrt(tf.squared_difference(features['x'],features['y']))
    
    return features

### b. data pipeline input function

In [5]:
def csv_input_fn(file_names, mode=tf.estimator.ModeKeys.EVAL, 
                 skip_header_lines=0, 
                 num_epochs=None, 
                 batch_size=200):
    
    shuffle = True if mode == tf.estimator.ModeKeys.TRAIN else False
    
    print("")
    print("* data input_fn:")
    print("================")
    print("Input file(s): {}".format(file_names))
    print("Batch size: {}".format(batch_size))
    print("Epoch Count: {}".format(num_epochs))
    print("Mode: {}".format(mode))
    print("Shuffle: {}".format(shuffle))
    print("================")
    print("")

    dataset = data.TextLineDataset(filenames=file_names)
    dataset = dataset.skip(skip_header_lines)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda csv_row: parse_csv_row(csv_row))
    dataset = dataset.map(lambda features, target: (process_features(features), target))
    
    #dataset = dataset.batch(batch_size) ??? very long time
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()
    
    features, target = iterator.get_next()
    return features, target

In [6]:
features, target = csv_input_fn(file_names=train_data_files)
print("Feature read from CSV: {}".format(list(features.keys())))
print("Target read from CSV: {}".format(target))


* data input_fn:
Input file(s): ['data/train-data.csv']
Batch size: 200
Epoch Count: None
Mode: eval
Shuffle: False

Feature read from CSV: ['x', 'y', 'alpha', 'beta', 'x_2', 'y_2', 'xy', 'dist_xy']
Target read from CSV: Tensor("IteratorGetNext:8", shape=(?,), dtype=float32)


## 3. Define Feature Columns

In [7]:
def get_feature_columns():
    
    CONSTRUCTED_NUMERIC_FEATURES_NAMES = ['x_2', 'y_2', 'xy', 'dist_xy']
    all_numeric_feature_names = NUMERIC_FEATURE_NAMES.copy() 
    
    if preprocess:
        all_numeric_feature_names += CONSTRUCTED_NUMERIC_FEATURES_NAMES

    numeric_columns = {feature_name: tf.feature_column.numeric_column(feature_name)
                       for feature_name in all_numeric_feature_names}

    categorical_column_with_vocabulary = \
        {item[0]: tf.feature_column.categorical_column_with_vocabulary_list(item[0], item[1])
         for item in CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.items()}
        
    feature_columns = {}

    if numeric_columns is not None:
        feature_columns.update(numeric_columns)

    if categorical_column_with_vocabulary is not None:
        feature_columns.update(categorical_column_with_vocabulary)
        
    
    # apply feature extentions
    
    feature_columns['alpha_X_beta'] = tf.feature_column.crossed_column(
        [feature_columns['alpha'], feature_columns['beta']], 4)
    
    return feature_columns

feature_columns = get_feature_columns()
print("Feature Columns: {}".format(feature_columns))

Feature Columns: {'x': _NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'y': _NumericColumn(key='y', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'x_2': _NumericColumn(key='x_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'y_2': _NumericColumn(key='y_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'xy': _NumericColumn(key='xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'dist_xy': _NumericColumn(key='dist_xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'alpha': _VocabularyListCategoricalColumn(key='alpha', vocabulary_list=('ax01', 'ax02'), dtype=tf.string, default_value=-1, num_oov_buckets=0), 'beta': _VocabularyListCategoricalColumn(key='beta', vocabulary_list=('bx01', 'bx02'), dtype=tf.string, default_value=-1, num_oov_buckets=0), 'alpha_X_beta': _CrossedColumn(keys=(_VocabularyListCategoricalColumn(key=

## 4. Instantiate an Estimator

In [8]:
def create_estimator(run_config, hparams):
    
    feature_columns = list(get_feature_columns().values())
    
    dense_columns = list(
        filter(lambda column: isinstance(column, feature_column._NumericColumn),
               feature_columns
        )
    )

    categorical_columns = list(
        filter(lambda column: isinstance(column, feature_column._VocabularyListCategoricalColumn) |
                              isinstance(column, feature_column._BucketizedColumn),
                   feature_columns)
    )

    indicator_columns = list(
            map(lambda column: tf.feature_column.indicator_column(column),
                categorical_columns)
    )
    
    
    estimator = tf.estimator.DNNRegressor(
        
        feature_columns= dense_columns + indicator_columns ,
        hidden_units= hparams.hidden_units,
        
        optimizer= tf.train.AdamOptimizer(),
        activation_fn= tf.nn.elu,
        dropout= hparams.dropout_prob,
        
        config= run_config
    )

    print("")
    print("Estimator Type: {}".format(type(estimator)))
    print("")
    
    return estimator

## 5. Define Serving Funcion

## 5. Run Experiment

In [9]:
def csv_serving_input_fn():
    
    SERVING_HEADER = ['x','y','alpha','beta']
    SERVING_HEADER_DEFAULTS = [[0.0], [0.0], ['NA'], ['NA']]

    rows_string_tensor = tf.placeholder(dtype=tf.string,
                                         shape=[None],
                                         name='csv_rows')
    
    receiver_tensor = {'csv_rows': rows_string_tensor}

    row_columns = tf.expand_dims(rows_string_tensor, -1)
    columns = tf.decode_csv(row_columns, record_defaults=SERVING_HEADER_DEFAULTS)
    features = dict(zip(SERVING_HEADER, columns))

    return tf.estimator.export.ServingInputReceiver(
        process_features(features), receiver_tensor)

### a. Define Experiment Function

In [10]:
def generate_experiment_fn(**experiment_args):

    def _experiment_fn(run_config, hparams):

        train_input_fn = lambda: csv_input_fn(
            train_data_files,
            mode = tf.contrib.learn.ModeKeys.TRAIN,
            num_epochs=hparams.num_epochs,
            batch_size=hparams.batch_size
        )

        eval_input_fn = lambda: csv_input_fn(
            valid_data_files,
            mode=tf.contrib.learn.ModeKeys.EVAL,
            num_epochs=1,
            batch_size=hparams.batch_size
        )

        estimator = create_estimator(run_config, hparams)

        return tf.contrib.learn.Experiment(
            estimator,
            train_input_fn=train_input_fn,
            eval_input_fn=eval_input_fn,
            eval_steps=None,
            #min_eval_frequency=1,
            #eval_hooks=[validation_monitor],
            **experiment_args
        )

    return _experiment_fn

### b. Set HParam and RunConfig

In [11]:
hparams  = tf.contrib.training.HParams(
    num_epochs = 1000,
    batch_size = 500,
    hidden_units=[8, 4], 
    dropout_prob = 0.0)

model_dir = 'trained_models/{}'.format(model_name)

run_config = tf.contrib.learn.RunConfig(
    save_checkpoints_steps=2400,
    tf_random_seed=19850610,
    model_dir=model_dir
)

print(run_config.model_dir)

trained_models/reg-model-03


### c. Run Experiment via learn_runner

In [12]:
if not resume:
    print("Removing previous artifacts...")
    shutil.rmtree(model_dir, ignore_errors=True)
else:
    print("Resuming training...") 

if train:
    tf.logging.set_verbosity(tf.logging.INFO)
    
    time_start = datetime.utcnow() 
    print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
    print(".......................................") 
    
    
    learn_runner.run(
        experiment_fn=generate_experiment_fn(

            export_strategies=[make_export_strategy(
                csv_serving_input_fn,
                exports_to_keep=1
            )]
        ),
        run_config=run_config,
        schedule="train_and_evaluate",
        hparams=hparams
    )
    
    time_end = datetime.utcnow() 
    print(".......................................")
    print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
    print("")
    time_elapsed = time_end - time_start
    print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))
    

Removing previous artifacts...
Experiment started at 15:21:43
.......................................
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11d8c4b70>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': 19850610, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 2400, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'trained_models/reg-model-03'}

Estimator Type: <class 'tensorflow.python.estimator.canned.dnn.DNNRegressor'>

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.

* data input_fn:
Input file(s): ['data/train-data.csv']
Batch size: 500
Epoch

INFO:tensorflow:Validation (step 4801): average_loss = 175.715, loss = 87857.3, global_step = 4801
INFO:tensorflow:loss = 76106.6, step = 4801 (2.211 sec)
INFO:tensorflow:global_step/sec: 62.8026
INFO:tensorflow:loss = 79618.1, step = 4901 (0.705 sec)
INFO:tensorflow:global_step/sec: 136.173
INFO:tensorflow:loss = 82839.5, step = 5001 (0.735 sec)
INFO:tensorflow:global_step/sec: 151.318
INFO:tensorflow:loss = 92482.0, step = 5101 (0.661 sec)
INFO:tensorflow:global_step/sec: 128.91
INFO:tensorflow:loss = 78531.5, step = 5201 (0.777 sec)
INFO:tensorflow:global_step/sec: 125.743
INFO:tensorflow:loss = 72378.3, step = 5301 (0.793 sec)
INFO:tensorflow:global_step/sec: 129.1
INFO:tensorflow:loss = 78896.2, step = 5401 (0.776 sec)
INFO:tensorflow:global_step/sec: 127.221
INFO:tensorflow:loss = 84309.2, step = 5501 (0.785 sec)
INFO:tensorflow:global_step/sec: 115.074
INFO:tensorflow:loss = 77246.3, step = 5601 (0.869 sec)
INFO:tensorflow:global_step/sec: 156.631
INFO:tensorflow:loss = 86233.2,

INFO:tensorflow:loss = 48634.8, step = 11801 (0.667 sec)
INFO:tensorflow:global_step/sec: 149.431
INFO:tensorflow:loss = 50012.7, step = 11901 (0.670 sec)
INFO:tensorflow:Saving checkpoints for 12001 into trained_models/reg-model-03/model.ckpt.
INFO:tensorflow:global_step/sec: 80.8172

* data input_fn:
Input file(s): ['data/valid-data.csv']
Batch size: 500
Epoch Count: 1
Mode: eval
Shuffle: False

INFO:tensorflow:Starting evaluation at 2017-11-03-15:23:21
INFO:tensorflow:Restoring parameters from trained_models/reg-model-03/model.ckpt-12001
INFO:tensorflow:Finished evaluation at 2017-11-03-15:23:22
INFO:tensorflow:Saving dict for global step 12001: average_loss = 116.236, global_step = 12001, loss = 58117.9
INFO:tensorflow:Validation (step 12001): average_loss = 116.236, loss = 58117.9, global_step = 12001
INFO:tensorflow:loss = 57615.9, step = 12001 (1.989 sec)
INFO:tensorflow:global_step/sec: 70.4052
INFO:tensorflow:loss = 56195.6, step = 12101 (0.669 sec)
INFO:tensorflow:global_step

INFO:tensorflow:loss = 42027.9, step = 18201 (0.852 sec)
INFO:tensorflow:global_step/sec: 135.032
INFO:tensorflow:loss = 48112.1, step = 18301 (0.736 sec)
INFO:tensorflow:global_step/sec: 138.825
INFO:tensorflow:loss = 42511.2, step = 18401 (0.723 sec)
INFO:tensorflow:global_step/sec: 131.165
INFO:tensorflow:loss = 44001.6, step = 18501 (0.759 sec)
INFO:tensorflow:global_step/sec: 149.306
INFO:tensorflow:loss = 44634.1, step = 18601 (0.670 sec)
INFO:tensorflow:global_step/sec: 155.774
INFO:tensorflow:loss = 42562.8, step = 18701 (0.642 sec)
INFO:tensorflow:global_step/sec: 155.421
INFO:tensorflow:loss = 43095.0, step = 18801 (0.644 sec)
INFO:tensorflow:global_step/sec: 155.847
INFO:tensorflow:loss = 33718.4, step = 18901 (0.642 sec)
INFO:tensorflow:global_step/sec: 154.152
INFO:tensorflow:loss = 31552.5, step = 19001 (0.648 sec)
INFO:tensorflow:global_step/sec: 156.138
INFO:tensorflow:loss = 42572.1, step = 19101 (0.641 sec)
INFO:tensorflow:Saving checkpoints for 19201 into trained_mod

## 7. Evaluate the Model

In [33]:
train_size = 12000
valid_size = 3000
test_size = 5000

train_input_fn = lambda: csv_input_fn(file_names= train_data_files, 
                                      mode= tf.estimator.ModeKeys.EVAL,
                                      batch_size= train_size)

valid_input_fn = lambda: csv_input_fn(file_names= valid_data_files, 
                                      mode= tf.estimator.ModeKeys.EVAL,
                                      batch_size= valid_size)

test_input_fn = lambda: csv_input_fn(file_names= test_data_files, 
                                      mode= tf.estimator.ModeKeys.EVAL,
                                      batch_size= test_size)

estimator = create_estimator(run_config, hparams)

train_results = estimator.evaluate(input_fn=train_input_fn, steps=1)
train_rmse = round(math.sqrt(train_results["average_loss"]),5)
print()
print("############################################################################################")
print("# Train RMSE: {} - {}".format(train_rmse, train_results))
print("############################################################################################")

valid_results = estimator.evaluate(input_fn=valid_input_fn, steps=1)
valid_rmse = round(math.sqrt(valid_results["average_loss"]),5)
print()
print("############################################################################################")
print("# Valid RMSE: {} - {}".format(valid_rmse,valid_results))
print("############################################################################################")

test_results = estimator.evaluate(input_fn=test_input_fn, steps=1)
test_rmse = round(math.sqrt(test_results["average_loss"]),5)
print()
print("############################################################################################")
print("# Test RMSE: {} - {}".format(test_rmse, test_results))
print("############################################################################################")

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11d8c4b70>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': 19850610, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 2400, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'trained_models/reg-model-03'}

Estimator Type: <class 'tensorflow.python.estimator.canned.dnn.DNNRegressor'>


* data input_fn:
Input file(s): ['data/train-data.csv']
Batch size: 12000
Epoch Count: None
Mode: eval
Shuffle: False

INFO:tensorflow:Starting evaluation at 2017-11-03-16:00:04
INFO:tensorflow:Restoring parameters from trained_models/reg-model-03/model.ckpt-24000
IN

## 8. Prediction

In [18]:
import itertools

predict_input_fn = lambda: csv_input_fn(file_names=test_data_files, 
                                      mode= tf.estimator.ModeKeys.PREDICT,
                                      batch_size= 5)

predictions = estimator.predict(input_fn=predict_input_fn)
values = list(map(lambda item: item["predictions"][0],list(itertools.islice(predictions, 5))))
print()
print("Predicted Values: {}".format(values))


* data input_fn:
Input file(s): ['data/test-data.csv']
Batch size: 5
Epoch Count: None
Mode: infer
Shuffle: False

INFO:tensorflow:Restoring parameters from trained_models/reg-model-03/model.ckpt-24000

Predicted Values: [51.184494, -4.3645005, 19.091375, 2.6998343, 1.6345019]


## What can we improve?

* **Use .tfrecords files instead of CSV** - TFRecord files are optimised for tensorflow.


* **Build a Custom Estimator** -  Custom Estimator APIs give you the flexibility to build custom models in a simple and standard way

